In [78]:
from decimal import *
from random import randint

chord = {
    'C':(0, 4, 7),
    'Cmaj7':(0, 4, 7, 11),
    'Dm':(2, 5, 9),
    'Dm7':(2, 5, 9, 12),
    'Em':(4, 7, 11),
    'Em7':(4, 7, 11, 14),
    'F':(5, 9, 12),
    'Fmaj7':(5, 9, 12, 16),
    'G':(7, 11, 14),
    'G7':(7, 11, 14, 17),
    'Am':(9, 12, 16),
    'Am7':(9, 12, 16, 19),
    'Bb':(10, 14, 17),
    'Bdim':(11, 14, 17)
}
    

In [2]:
def nextChord(beatSum, indexAcc, curChord):
    indexAcc += 1
    curChord = chordProg[indexAcc]
    diff = beatSum-Decimal(0.5)
    if diff >= 0.5:
        beatSum, indexAcc, curChord = nextChord(diff, indexAcc, curChord)
    else:
        beatSum = diff
    return beatSum, indexAcc, curChord

In [161]:
def varchord(re, curChord, randomInt):
#     basicchord=60
#     highchord=72
#     lowchord=48
    if randomInt==0:
        for i, component in enumerate(chord[curChord]):
            re += str(60+component) + ','
#         return basicchord
    elif randomInt==1:
        for i, component in enumerate(chord[curChord]):
            if i==0:
                re += str(60+component+12) + ','
            else:
                re += str(60+component) + ','
#         return highchord
    elif randomInt==2:
        for i, component in enumerate(chord[curChord]):
            if i==0:
                re += str(60+component-12) + ','
            else:
                re += str(60+component) + ','
#        return lowchord
    elif randomInt==3:
        for i, component in enumerate(chord[curChord]):
            if i==1:
                re += str(60+component+12) + ','
            elif i==0:
                re += str(60+component+12) + ','
            else:
                re += str(60+component) + ','
#        第一個音和第二個音同時增加
    elif randomInt==4:
        for i, component in enumerate(chord[curChord]):
            if i==1:
                re += str(60+component-12) + ','
            elif i==0:
                re += str(60+component-12) + ','
            else:
                re += str(60+component) + ','
#       第一個音和第二個音同時減少
#         basichord
#     return basichord,highchord,lowchord;
    return re

In [162]:
def accompanyMelody(staff, chordProg):
    staffReturn = {}
    for key in sorted(staff.keys()):
        indexAcc = 0
        curChord = chordProg[indexAcc]
        beatSum = 0
        re = ''
        totalBeatSum = 0
        for beat in staff[key].split(';'):
            duration = beat.split(',')[0]
            noteOrRest = beat.split(',')[1]

            #Determine when to select next chord
            if beatSum >= 0.5:
                beatSum, indexAcc, curChord = nextChord(beatSum, indexAcc, curChord)

            #Generate different strings according to note or rest
            
            if noteOrRest == '1':
                re += duration + ','
                
                randomInt = randint(0,4)
                re = varchord(re, curChord, randomInt) #最後第三個參數放時randint(0,4)須能正確執行，結果正確
                
                re = re[:-1] + ';'
            else:
                re += duration + ',0;'

            #Add 0.0000000000001 to beatSum while it's a tuplet beat
            if Decimal(duration) * 128 % 1 == 0:
                beatSum += Decimal(duration)
            else:
                beatSum += Decimal(duration) + Decimal(0.0000000000001)

        staffReturn[key] = re[:-1]
    return staffReturn

In [174]:
chordProg = ['C','G','Am','Em','F','Cmaj7','Dm','G7','C','G','Am','Em','F','Cmaj7','Dm','G7']

staff = {
    '100':'1,1;0.5,1;0.5,1;1,1;0.5,1;0.5,1;1,1;0.5,1;0.5,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1',
#     '101':'0.4895833333333333333333333333,0;0.01041666666666666666666666667,1;0.5,1;0.25,1;0.125,1;0.125,1;0.125,1;0.25,1;0.125,1;0.25,1;0.0625,1;0.0625,1;0.0625,1;0.0625,1;0.125,1;0.125,1;0.125,1;0.125,1;0.625,0;0.375,1;0.4895833333333333333333333333,0;0.01041666666666666666666666667,1;0.5,1;0.25,1;0.125,1;0.125,1;0.125,1;0.25,1;0.125,1;0.25,1;0.0625,1;0.0625,1;0.0625,1;0.0625,1;0.125,1;0.125,1;0.125,1;0.125,1;0.625,0;0.375,1'
}

test = accompanyMelody(staff, chordProg)

for key in test:
    for element in test[key].split(';'):
        print element
    print

1,48,52,67
0.5,69,72,76
0.5,76,79,71
1,77,81,72
0.5,74,65,69
0.5,55,71,74,77
1,72,64,67
0.5,69,72,76
0.5,64,67,71
0.125,77,81,72
0.125,53,69,72
0.125,77,81,72
0.125,65,69,72
0.125,72,76,67,71
0.125,72,76,67,71
0.125,48,64,67,71
0.125,60,64,67,71
0.125,74,65,69
0.125,74,65,69
0.125,50,53,69
0.125,50,65,69
0.125,67,71,74,77
0.125,79,83,74,77
0.125,67,71,74,77
0.125,79,83,74,77



1,60,64,67
0.5,69,72,76
0.5,64,67,71
1,65,69,72
0.5,62,65,69
0.5,67,71,74,77
1,60,64,67
0.5,69,72,76
0.5,64,67,71
0.125,65,69,72
0.125,65,69,72
0.125,65,69,72
0.125,65,69,72
0.125,60,64,67,71
0.125,60,64,67,71
0.125,60,64,67,71
0.125,60,64,67,71
0.125,62,65,69
0.125,62,65,69
0.125,62,65,69
0.125,62,65,69
0.125,67,71,74,77
0.125,67,71,74,77
0.125,67,71,74,77
0.125,67,71,74,77

